<a href="https://colab.research.google.com/github/RussAbbott/TicTacToe/blob/master/Reinforcement_learning_for_tic_tac_toe_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning for tic-tac-toe

Based on Daniel Sauble's [article](https://medium.com/swlh/tic-tac-toe-and-deep-neural-networks-ea600bc53f51) and Github [repo](https://github.com/djsauble/tic-tac-toe-ai/blob/master/reinforcement_learning_for_tic-tac-toe.ipynb).

In [1]:
#@title Imports

import random
import numpy as np
# import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.backend import reshape
from keras.utils.np_utils import to_categorical

In [2]:
#@title class Config

class Config:
    '''
    Static information and methods about the game
    '''
    def __init__(self, board_side=3):
        self.board_side = board_side
        self.triples = self.collect_the_triples()
        self.x = 'X'
        self.o = 'O'
        self.empty = ' '
        self.draw = 'Draw'

    # v v v v v v v v v v v v v v v v v v staticmethod v v v v v v v v v v v v v v v v v v 

    @staticmethod
    def tuple_board(board):
        '''
        Convert a 3x3 board into a 3-tuple of 3-tuples
        '''
        list_of_tuples = [tuple(row.tolist()) for row in board]
        return tuple(list_of_tuples)

    # ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ staticmethod ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 


    def collect_the_triples(self):
        '''
        Generate all 3-in-a-row cell sequences
        Each cell sequence will be returned as [(r, c), (r, c), (r, c)],
        where the (r, c)'s are the cell coordinates. (Each cell will appear in a number of sequences.)
        '''
        row_triples = [[(r, c) for c in range(self.board_side)] for r in range(self.board_side)]
        col_triples = [[(r, c)  for r in range(self.board_side)] for c in range(self.board_side)]
        pos_diag = [(rc, rc) for rc in range(self.board_side)]                  # [(0, 0), (1, 1), (2, 2)]]
        neg_diag = [(r, (self.board_side - 1) - r) for r in range(self.board_side)]  # [(0, 2), (1, 1), (2, 0)]]
                        
        all_triples = row_triples + col_triples + [pos_diag, neg_diag]
        return all_triples


    def new_board(self):
        board = np.full( (self.board_side, self.board_side), self.empty, dtype='str')
        return board


    def normalize_board_state(self, board, board_state_mapping, board_states_seen):
        '''
        Input: a 3x3 board state
        Output: the distinguished board state that represents the input board state
        First check to see if the input board state already has an associated
        distinguished board state. If so, return that distinguished board state.
        If not, generate all 8 equivalent board states. Select one of the, arbitrarily
        the smallest under < . Point all 8 board states to the distiguished board state.
        Do all this with tuples because dictionaries require immutable keys.
        '''
        t_board = self.tuple_board(board)
        # Include the tuple version of the board to board_states_seen
        board_states_seen.add(t_board)
        n_board = board_state_mapping.get(t_board)
        if n_board is not None:
            return n_board

        t_boards = []
        for _ in range(4):
            t_boards.append(t_board)
            t_boards.append(self.tuple_board(np.transpose(t_board)))
            t_board = self.tuple_board(np.rot90(t_board))
        min_board = min(t_boards)

        for t_board in t_boards:
            if t_board not in board_state_mapping:
                board_state_mapping[t_board] = min_board

        return min_board





In [3]:
#@title class Game

class Game:
    '''
    The dynamic information about the game along wihth link to Config()
    '''
    def __init__(self, x_strategy=None, o_strategy=None, verbose=False):
        self.config = Config(board_side=3)
        self.board = self.config.new_board()
        self.board_history = []
        self.active_mark = self.config.x
        self.other_mark = self.config.o
        self.active_strategy = x_strategy
        self.other_strategy = o_strategy
        self.verbose = verbose

        # # For minimax
        # self.depth_limit = None
        # self.depth = 0


    # v v v v v v v v v v v v v v v v v v staticmethod v v v v v v v v v v v v v v v v v v 

    # ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ staticmethod ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 


    def game_status(self, requested_counts={3: 0, 2: 0, 1: 0}):
        '''
        returns: game_over, counts

        The counts are dictionary items() of numbers of triples with exactly 1, 2, or 3 of a kind with the rest blanks.
        The o's counts are counted negatively to facilitate minimax.
        '''
        triples = self.config.triples
        counts = requested_counts.copy()
        any_triples_in_play = False
        x = self.config.x
        o = self.config.o


        for triple in triples:
            triple_with_values = [self.board[r][c] for (r, c) in triple]
            xs_in_triple, os_in_triple = triple_with_values.count(x), triple_with_values.count(o)

            # A triple is in play if it has no more than one player and that player has not won the triple
            
            # The following is about the |= (in-place 'or') operation.
            # |= does not short-circuit even if the LHS is already true. That is,
            #                   x |= expression
            # will evaluate expression even if x is already true. Hence:
            if not any_triples_in_play:
                any_triples_in_play = ( 
                                        (xs_in_triple == 0 or os_in_triple == 0) and
                                        (xs_in_triple + os_in_triple < 3)
                                      )

            for count in requested_counts:
                if xs_in_triple == count and os_in_triple == 0: 
                    counts[count] += 1

                # Count the opponents scores negatively
                if os_in_triple == count and xs_in_triple == 0: 
                    counts[count] -= 1

        game_over = counts[3] != 0 or not any_triples_in_play

        return game_over, list(counts.items())


    def get_winner(self, counts):
        '''
        Called when the game is known to be over. Returns either config.x or config.o if there is
        a winner, or config.draw if the game is a draw.
        '''
        winner = self.config.x if counts[3] > 0 else self.config.o if counts[3] < 0 else self.config.draw
        return winner

        
    def moves_to_board(self, moves, verbose=False):
        '''
        Reconstruct the board from a list of moves.
        This is run in a new and independent Game
        '''
        for player, (row, col) in moves:
            self.board[row, col] = player
            if verbose:
                game.print_board()
                print('\n')


    def print_board(self, title=''):
        '''
        Print either the current board or a supplied board
        '''
        # player_symbol = config.player_symbol
        rows = 'ABC'
        print(f'\n{title}')
        print('    1   2   3')
        for r in range(len(self.board)):
            print(f' {rows[r]} ', end='')
            for c in range(len(self.board[r])):
            #     mark = player_symbol[board[r][c]]
                mark = ' ' + self.board[r][c] + ' '
                last_in_row = c == len(self.board[r]) - 1
                print(f'{mark}{"" if last_in_row else "|"}', end='\n' if last_in_row else '')
            if (r < len(self.board) - 1):
                print("   -----------")

        game_over, counts = self.game_status()
        if game_over:
            winner = self.get_winner(dict(counts))
            caption = '     ' + ((winner + " wins") if winner in [self.config.x, self.config.o] else (' ' + winner))
            dots = "   ..........."
            print(dots)
            print(caption)
            print(dots)

    def switch_active_player(self):
        # This does the right thing. The RHS is fully evaluated before assignment to the LHS.
        self.active_mark, self.other_mark = self.other_mark, self.active_mark
        self.active_strategy, self.other_strategy = self.other_strategy, self.active_strategy


In [ ]:
# @title Test the Game class

game = Game()

x = game.config.x
o = game.config.o

history = [(x, (0, 0)), (x, (1, 1)), (x, (2, 2)), # X wins
           (o, (2, 2)), (o, (1, 2)), (o, (0, 2)), # O wins
           (x, (1, 2)), (o, (1, 0)), (x, (2, 1)), (o, (0, 1)), # Draw
           ]

game.moves_to_board(history, verbose=True)


In [25]:
# @title class PlayableGame

class PlayableGame(Game):

    def __init__(self, x_strategy=None, o_strategy=None, verbose=False):
        # For minimax
        self.depth_limit = None
        self.depth = 0
        super().__init__(x_strategy=x_strategy, o_strategy=o_strategy, verbose=verbose)
        
    # v v v v v v v v v v v v v v v v v v staticmethod v v v v v v v v v v v v v v v v v v 

    @staticmethod
    def cell_label(cell):
        '''
        Convert a zero-based (row, col) pair of coordinates to Letter-Number notation.
        E.g., (0, 1) -> A2
        '''
        r, c = cell
        return ['A', 'B', 'C'][r](c+1)
                
                
    @staticmethod
    def explain_move_input():
        print(f'Indicate a move as follows.\n')
        print('   1     2    3')
        print('A  A1 |  A2 | A3')
        print('  ---------------')
        print('B  B2 |  B2 | B3')
        print('  ---------------')
        print('C  C3 |  C2 | C3')
        print('Lower case a, b, c may be used.\n')


    @staticmethod
    def print_game_heading(x_strategy, o_strategy):
        title = f'{x_strategy.__name__} (X) vs {o_strategy.__name__} (O)'
        len_title = len(title)
        dashes = '- '*(10 + len_title//2 + (1 if len_title%2==1 else 0))
        print(f'\n\n' + dashes)
        print(f'{" "*10}{title}')
        print(dashes + '\n')

    # ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ staticmethod ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 


    def child_board(self, move):
        '''
        Put self.active_mark at (r, c) of a copy of board.
        Return the updated board, but don't change self.board.
        '''
        next_board = self.board.copy()
        r, c = move
        next_board[r][c] = self.active_mark
        return next_board


    def game_stats(self, x_strategy, o_strategy, nbr_games, games):
        '''
        Aggregate win/loss/draw stats for a player
        Input: a list of (move_history, board_history) pairs.
        We ignore the move_history and look only at the final board_history
        '''
        if isinstance(x_strategy, Sequential):
            x_strategy = self.model_strategy 

        if isinstance(o_strategy, Sequential):
            o_strategy = self.model_strategy 

        print(f'X ({x_strategy.__name__}) vs O ({o_strategy.__name__}). {nbr_games} games.')

        # Keep the stats from the perspective of the x player
        stats = {"x": 0, "o": 0, "draw": 0}
        for board_history in games:
            _status, counts = self.game_status(board_history[-1])
            counts = dict(counts)
            if counts[3] > 0:
                stats["x"] += 1
            elif counts[3] < 0:
                stats["o"] += 1
            else:
                stats["draw"] += 1
        
        player_x_pct = stats["x"] / len(games) * 100
        player_o_pct = stats["o"] / len(games) * 100
        draw_pct = stats["draw"] / len(games) * 100

        print("Player\twins\tpct wins")
        print("------\t----\t--------")
        print(f"   x\t {stats['x']}\t  {round(player_x_pct)}%")
        print(f"   o\t {stats['o']}\t  {round(player_o_pct)}%")
        print(f" Draws\t {stats['draw']}\t  {round(draw_pct)}%")
        print()


    def get_user_move(self):
        '''
        Get the user's input and translate to zero-based (row, col)
        '''
        char_to_nbr = {'A': 0, 'B': 1, 'C': 2, 'a': 0, 'b': 1, 'c': 2, '1': 0, '2': 1, '3': 2}
        
        # Loop until the user produces a valid input. Return from within the loop.
        while True:
            strng = input(f'\n{self.active_mark}\'s move > ')
            chars = [char_to_nbr[c] for c in strng if c in char_to_nbr]

            if len(chars) != 2:
                print('Invalid input')
                continue

            r, c = chars 
            cell = self.board[r][c]
            # print(f'{strng = } {chars = } {(r, c)} {cell = } {self.config.empty = } {cell == self.config.empty = } ')
            if cell == self.config.empty:
                # print(f'Returning {(r, c)}')
                return r, c
            else:
                print(f'Cell {strng} already contains {cell}\n')
                

    def human_strategy(self):
        '''
        Get the user's row-col move in the form of A1, A2, A3, B1, B2, B3, C1, C2, C3.
        A, B, C identifies the row; the number identifies the column.
        Translate that to zero-based (row, col).
        '''
        print()
        self.print_board()
        r, c = self.get_user_move()
        return r, c


    def minimax_moves_and_score(self):
        '''
        Because the board and the active player change frequently, they are passed explicitly to the
        minimax methods--rathan using the board and active_mark stored a self-values. 

        Given a board and given that this isn't a terminal node, use minimax to find and return:
            (a) the minimax score for this board along with 
            (b) a list of moves that produce it.
        Note: board may be down the tree from the current board stored at self.board.
        '''
        v_moves = self.valid_moves()
        # For each valid move, find the minimax score for the board that that move produces.
        # Use self.wrap_run_restore() to:
        # (a) update the state, (b) run minimax_score() to find the minimax value, and (c) restore the state
        switched_mark = self.config.x if self.active_mark == self.config.o else self.config.o 
        move_scores = [(move, self.wrap_run_restore(fn=self.minimax_score, 
                                                    next_depth=self.depth+1, 
                                                    next_board=self.child_board(move), 
                                                    next_mark=switched_mark))
                      for move in v_moves]
        # move_scores = [(move, self.minimax_score(depth-1, self.child_board(self.board, move, switched_mark)))
        #               for move in v_moves]
        # The player playing X is the maximizer.
        is_maximizer = self.active_mark == self.config.x
        min_or_max = max if is_maximizer else min
        # Initially, we ignore the specific move that prodocued the best score.
        _best_move, best_score = min_or_max(move_scores, key=lambda m_s: m_s[1])


        # If we are at the top level, print the various moves and the scores they produce.
        # Mark the ones that are the best score.
        if self.verbose and self.depth == 0:
            # print(f"\n\nplayer = {config.player_symbol[player][1]}.")
            self.print_board(title=f"\n\n{self.active_strategy = }.")
            print()
            for move, score in move_scores:
                # move is the (row, col) pair to be occupied by this move
                # Since (row, col) is zero-based, translate to the notation used for human play.
                print(f"{self.cell_label(move)} -> {score} {' (ok)' if score == best_score else ''}")
            print(f"{best_moves = }")

        # Find all the moves that produce that best score.
        best_moves = [move for move, score in move_scores if score == best_score]

        return best_moves, best_score


    def minimax_score(self):
        '''
        Returns the minimax score for board.
        '''
        # Evaluate the game at its current state
        game_over, score = self.game_status()
        # If at a terminal node, i.e., if self.depth = self.depth_limit or game_over, 
        # keep this score and return it as the score value.
        # Otherwise,
        if self.depth != self.depth_limit and not game_over:
            # call minimax recursively to get the score.    
            _moves, score = self.minimax_moves_and_score()
        
        return score


    def minimax_strategy(self, depth_limit):
        # If minimax is playing X, select a corner as a first move.
        # (This is hard-wired. The scoring algorithm would select the center square.)
        if np.count_nonzero(self.board==self.config.empty) == 9:
            return random.choice([(0, 0), (0, 2), (2, 0), (2, 2)]) 

        self.depth_limit = depth_limit
        moves, _score = self.minimax_moves_and_score()
        move = random.choice(moves)
        return move


    def model_strategy(self, model, rnd=0):
        '''
        Use the model to get the best next move for the given player at the given board position
        '''
        scores = []
        v_moves = self.valid_moves(game.board, config.empty)
        # Make predictions for each possible move
        if self.verbose:
            print('\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        for row, col in v_moves:
            # row, col = v_moves[i]
            next_board = np.array(game.board)
            next_board[row][col] = game.active_mark
            next_board_1d = next_board.reshape((-1, 9))
            prediction = model.predict(next_board_1d)[0]
            np.around(prediction, 2, out=prediction)
            if self.verbose:
                self.print_board(next_board, config, title=f'If {self.active_mark} moves to {self.cell_label((row, col))}:')
                for label, prob in zip(["Draw", "X wins", "O wins"], prediction):
                    print(f'{label} = {prob}')

            if game.active_mark == self.config.x:
                draw_prediction, win_prediction, loss_prediction = prediction
            elif game.active_mark == self.config.o:
                draw_prediction, loss_prediction, win_prediction = prediction
            else:
                message = f'The active_mark ({self.active_mark}) is neither {self.config.x} nor {self.config.o}.\n' 
                raise Exception(message)

            if win_prediction > loss_prediction:
                scores.append(win_prediction - loss_prediction)
            else:
                scores.append(draw_prediction - loss_prediction)

        # Choose the best move with a random factor
        best_moves_indices = np.flip(np.argsort(scores))
        move = None
        for i in range(len(best_moves_indices)):
            if random.random() * rnd < 0.5:
                move = v_moves[best_moves_indices[i]]

        if move == None:
            # Choose a move at random
            move = random.choice(v_moves)

        if self.verbose:
            print(f'\n==> {self.active_mark} moves to {move}.')
            print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')

        return move


    def play_a_game(self, x_strategy, o_strategy, verbose=False, rnd=0):
        '''
        Play a game.
        '''
        super().__init__(x_strategy, o_strategy, verbose=verbose)

        human_is_playing = self.human_strategy in [x_strategy, o_strategy]

        if human_is_playing:
            self.print_game_heading(x_strategy, o_strategy)
            # print(f'\n\n{game.active_strategy.__name__} (X) vs {game.other_strategy.__name__} (O)\n\n')
            self.explain_move_input()

        # game_status returns (status, score), where status is True/False depending on whether the game is over. 
        # So, the followinhg line says: repeat this loop as long as status is not True, i.e., game_over.
        while not self.game_status()[0]:

            row, col = self.select_a_move(rnd=rnd)
            # Make the move
            self.board[row][col] = self.active_mark

            # Add the board to board_history
            self.board_history.append(self.board.copy())
            
            # Switch the active player
            self.switch_active_player()

        if human_is_playing:
            self.print_board(title='\n    Game over\n   ...........')
                    

    def random_strategy(self):
        '''
        Return a random (valid) move
        '''
        v_moves = self.valid_moves()
        move = random.choice(v_moves)
        return move


    def select_a_move(self, rnd=0):
        '''
        Let self.active_strategy select a move.
        '''
        if isinstance(self.active_strategy, Sequential):
            # In this case, self.active_strategy is the trained model rather than a "strategy.". 
            # Pass the model to model_strategy().
            model = self.active_strategy
            return self.model_strategy(model, rnd=rnd)

        if self.active_strategy == self.human_strategy:
            return self.human_strategy()

        if self.active_strategy in [self.minimax_strategy, self.win_block_strategy]:
            depth = 1 if self.active_strategy == self.win_block_strategy else 5
            return self.minimax_strategy(depth)

        return self.random_strategy()


    def valid_moves(self):
        '''
        Get a list of valid moves.
        '''
        board_side = self.config.board_side
        empty = self.config.empty
        moves = [(r, c) for r in range(board_side) 
                        for c in range(board_side) 
                        if self.board[r][c] == empty]
        return moves


    def win_block_strategy(self, depth=1):
        '''
        The win_block_strategy is a minimax player limited to a search depth of 1.
        In other words, it looks ahead one move: what will its current move yield?
        
        This lets it determine whether it can win on its current move. 
        
        It also allows it to determine whether it is leaving the other player 
        2 in a row, which will let the other player win on its next move.
        '''
        return self.minimax_strategy(depth)

    
    def wrap_run_restore(self, fn, next_depth, next_board, next_mark):
        # Save the current depth, board and active_mark
        current_depth, current_board, current_mark = self.depth, self.board, self.active_mark

        # Update the state
        self.depth, self.board, self.active_mark = next_depth, next_board, next_mark

        # Call the function
        result = fn()

        # Restore the state
        self.depth, self.board, self.active_mark = current_depth, current_board, current_mark

        # Return the result
        return result





In [29]:
#@title test Playable_Game

random.seed()

game = PlayableGame()

for x_strategy, o_strategy in  [
                                # (game.random_strategy, game.random_strategy), 
                                # (game.random_strategy, game.human_strategy),  
                                # (game.human_strategy, game.random_strategy),
                                (game.random_strategy, game.win_block_strategy),
                                (game.win_block_strategy, game.minimax_strategy),
                                ]:
    game.play_a_game(x_strategy=x_strategy, o_strategy=o_strategy)

    if game.human_strategy not in [x_strategy, o_strategy]:
        game.print_game_heading(x_strategy, o_strategy)

        for game.board in game.board_history:
            game.print_board()

stop



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
          random_strategy (X) vs win_block_strategy (O)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



    1   2   3
 A    | X |   
   -----------
 B    |   |   
   -----------
 C    |   |   


    1   2   3
 A    | X |   
   -----------
 B    | O |   
   -----------
 C    |   |   


    1   2   3
 A    | X |   
   -----------
 B  X | O |   
   -----------
 C    |   |   


    1   2   3
 A    | X | O 
   -----------
 B  X | O |   
   -----------
 C    |   |   


    1   2   3
 A    | X | O 
   -----------
 B  X | O |   
   -----------
 C    |   | X 


    1   2   3
 A    | X | O 
   -----------
 B  X | O |   
   -----------
 C  O |   | X 
   ...........
     O wins
   ...........


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
          win_block_strategy (X) vs minimax_strategy (O)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



    1   2   3
 A  X |  

NameError: ignored

In [24]:
class B:

    def fn_b():
        class_a = A()
        print('fn_b: Calling class_a.fn_a1(class_a)')
        class_a.fn_a1(class_a)
        # print('fn_b: Calling A.fn_a1(class_a)')
        # A.fn_a1(class_a)

class A:

    @staticmethod
    def fn_a1(gme):
        print(f'\nfn_a1: {gme = }. Calling gme.fn_a2b()')
        gme.fn_a2b()
        print(f'\nfn_a1: {gme = }. Calling A.fn_a2b(gme)')
        A.fn_a2b(gme)

    def fn_a2a(self):
        print(f'fn_a2a: {self = }')

    def fn_a2b(gme):
        print(f'fn_a2b: {gme = }, Calling gme.fn_a3(gme.fn_a2a)')
        gme.fn_a3(gme.fn_a2a)

    @staticmethod
    def fn_a3(fn):
        print(f'fn_a3: {fn = }. Calling fn()')
        fn()

B.fn_b()

fn_b: Calling class_a.fn_a1(class_a)

fn_a1: gme = <__main__.A object at 0x7fc2cc271700>. Calling gme.fn_a2b()
fn_a2b: gme = <__main__.A object at 0x7fc2cc271700>, Calling gme.fn_a3(gme.fn_a2a)
fn_a3: fn = <bound method A.fn_a2a of <__main__.A object at 0x7fc2cc271700>>. Calling fn()
fn_a2a: self = <__main__.A object at 0x7fc2cc271700>

fn_a1: gme = <__main__.A object at 0x7fc2cc271700>. Calling A.fn_a2b(gme)
fn_a2b: gme = <__main__.A object at 0x7fc2cc271700>, Calling gme.fn_a3(gme.fn_a2a)
fn_a3: fn = <bound method A.fn_a2a of <__main__.A object at 0x7fc2cc271700>>. Calling fn()
fn_a2a: self = <__main__.A object at 0x7fc2cc271700>


In [ ]:
# @title  Generate a collection of simulated games to calculate win/lose statistics for first and second players--and later to train our neural network. 
# 'X' (player 1) should have an edge due to first mover advantage.


game = Game()
x_strategy, o_strategy, nbr_games = game.random_strategy, game.random_strategy, 1000
games_rr = [game.play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)] 
game_stats(x_strategy, o_strategy, nbr_games, games_rr)

x_strategy, o_strategy, nbr_games = game.win_block_strategy, game.random_strategy, 100
games_wr = [game.play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)] 
game_stats(x_strategy, o_strategy, nbr_games, games_wr)

x_strategy, o_strategy, nbr_games = game.random_strategy, game.win_block_strategy, 100
games_rw = [game.play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)] 
game_stats(x_strategy, o_strategy, nbr_games, games_rw)

games = games_rr + games_wr*10 + games_rw*10

In [ ]:
# @title Functions to normalize and label game states.

# def tuple_board(board):
#     '''
#     Convert a 3x3 board into a 3-tuple of 3-tuples
#     '''
#     list_of_tuples = [tuple(row.tolist()) for row in board]
#     return tuple(list_of_tuples)


# def normalize_board_state(board, board_state_mapping, board_states_seen):
#     '''
#     Input: a 3x3 board state
#     Output: the distinguished board state that represents the input board state
#     First check to see if the input board state already has an associated
#     distinguished board state. If so, return that distinguished board state.
#     If not, generate all 8 equivalent board states. Select one of the, arbitrarily
#     the smallest under < . Point all 8 board states to the distiguished board state.
#     Do all this with tuples because dictionaries require immutable keys.
#     '''
#     t_board = tuple_board(board)
#     # Include the tuple version of the board to board_states_seen
#     board_states_seen.add(t_board)
#     n_board = board_state_mapping.get(t_board)
#     if n_board is not None:
#         return n_board

#     t_boards = []
#     for _ in range(4):
#         t_boards.append(t_board)
#         t_boards.append(tuple_board(np.transpose(t_board)))
#         t_board = tuple_board(np.rot90(t_board))
#     min_board = min(t_boards)

#     for t_board in t_boards:
#         if t_board not in board_state_mapping:
#             board_state_mapping[t_board] = min_board

#     return min_board


def board_history_to_labelled_board_states(board_history, board_state_mapping, board_states_seen):
    '''
    Input: a list of board positions for a game.
    Output: a list of pairs, where each board position is paired with (labeled by) the winner of the game.
    In the code above, a win by o is indicated by -1; a draw is indicated by 0; a win by x is indicated by 1.
    However, we will use a neural net that maps board states to categories. Categories must be
    non-negative integers. So we map (-1, 0, 1) -> (2, 0, 1) for category designation. 
    '''

    # Get a list of normalizezd board states for the states in board_history. 
    # See normalize_board_state() for what that means.
    normalized_board_states = [normalize_board_state(board, board_state_mapping, board_states_seen) for board in board_history]

    # Get the winner of the board_history by looking at the final board in board_history.
    _status, counts_as_tuples = game_status(board_history[-1], config)
    winner = get_winner(dict(counts_as_tuples), config)
    
    # winner will be:config.x, config.o, or config.draw
    # convert the winner to: {draw: 0; x won: 1, o_won: 2}; 
    # In categorical learning, the categories must all be non-negative integers
    winner_to_category = {config.draw: 0, config.x: 1, config.o: 2}
    winner_category = winner_to_category[winner]

    # Make a list of the winner category as long as the board_history
    list_of_winner_category = [winner_category] * len(board_history)
    # Zip the two lists together and return the result.
    return zip(normalized_board_states, list_of_winner_category)


def labeled_and_shuffled_game_histories(games):
    '''
    Input: a list of (move_history, board_history) pairs, one for each game.

    Produce a collection of board states labelled by who eventually won that game.
    Return X y where X and y are lists of board states and the correspoding labels/categories.
    The variable names X and y are the names used traditionally. X is upper case.

    1. Ignore move_history. Work only with board_history
    2. Use the last element in board_history to determine the game winner.
    3. Pair each board position in board_history with that winner as a category/label.
    4. Append the list of board positions to X and a list of length len(board_history) of winner to y   
    5. Shuffle X and y simultaneously so that the labelled board posistions are spread around. 
    '''

    # Because of symmetry, for every board state there are 7 equivalent board states. 
    # Select one of the 8 to represent them all.
    # Store the mapping from board_state to distinguished board_state in the following cache (dictionary).
    # Also use the dictionary to store the count of distinct board states.
    board_state_mapping = {}
    board_states_seen = set()

    # Will be a list of (board_position, label) pair.
    shuffled_Xs_ys = []

    for _move_history, board_history in games:
        shuffled_Xs_ys_for_a_game = board_history_to_labelled_board_states(board_history, board_state_mapping, board_states_seen)
        shuffled_Xs_ys += shuffled_Xs_ys_for_a_game

    # Shuffle the (board_position, winner) pairs so that not all the winners/losers occur first.
    # random.shuffle shuffles a list in place
    random.shuffle(shuffled_Xs_ys)

    print(f'A simple upper bound for the size of the tic-tac-toe state space is {3**9 = }. ')
    print('(Three states for each cell and nine cells.) ')
    print('This count includes many illegal positions, such as a position with five Xs and no Os')
    print('or a position in which both players have a row of three.\n')
    print('A more careful count, removing these illegal positions, gives 5,478.')
    print('When rotations and reflections are considered, there are 765 different positions.')
    print('Wikipedia: https://en.wikipedia.org/w/index.php?title=Game_complexity&oldid=950763371#Example:_tic-tac-toe_(noughts_and_crosses)\n')

    print(f'Summary. Valid board states: 5478.  After considering symmetries: 765.\n')
    print(f'Board states seen: {len(board_states_seen)}. Number of distinguished boards: {len(set(board_state_mapping.values()))} ')

    return shuffled_Xs_ys



def unzip(zipped):
    [a, b] = zip(*zipped)
    return list(a), list(b)    
        
    


In [ ]:
# @title Label the board postions in the games abd shuffkle the labelled pairs.

shuffled_Xs_ys = labeled_and_shuffled_game_histories(games)

X, y = unzip(shuffled_Xs_ys)

X = np.array(X).reshape((-1, 9))
y = to_categorical(y, num_classes=3)

# Split out the train and test data
trainNum = int(len(X) * 0.8)
X_train, X_test, y_train, y_test = X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:]


In [ ]:
#@title Prepare the neural net.

def build_model(board_side):
    '''
    Create a NN model
    '''
    num_cells = board_side * board_side
    outcomes = 3 # The number of possible outcomes in a game. (draw, X-wins, O-wins)
    model = Sequential([
                        Dense(200, input_shape=(num_cells, ), activation='relu'),
                        Dropout(0.2),
                        Dense(125, activation='relu'),
                        Dense(75, activation='relu'),
                        Dropout(0.1),
                        Dense(25, activation='relu'),
                        Dense(outcomes, activation='softmax'),
                        ])
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    print(model.summary())    

    return model




In [ ]:
#@title Train the model on the tic-tac-toe games  generated earlier.

game = Game()
model = build_model(game.config.board_side)


# Train the model
n_epochs = 500
batch_size = 100

# history output is not used
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=n_epochs, batch_size=batch_size)
print('\n\tThe model is trained\n')


In [ ]:
# @title First play a random player against itself and then the model against a random player.

x_strategy = random_strategy
o_strategy = random_strategy
nbr_games = 1000
games_rr = [play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)]
game_stats(x_strategy, o_strategy, nbr_games, games_rr)

x_strategy = model
o_strategy = random_strategy
nbr_games = 100
games_mr = [play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)]
game_stats(x_strategy, o_strategy, nbr_games, games_mr)

x_strategy = random_strategy
o_strategy = model
nbr_games = 100
games_rm = [play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)]
game_stats(x_strategy, o_strategy, nbr_games, games_rm)

x_strategy = model
o_strategy = model 
nbr_games = 100
games_mm = [play_a_game(x_strategy=x_strategy, o_strategy=o_strategy) for _ in range(nbr_games)]
game_stats(x_strategy, o_strategy, nbr_games, games_mm)

* In the original the random player won 76% of games as x; the model won 98%

* In the original the random player won 48% of games as o; the model won 96%.




In [ ]:
x_strategy = human_strategy
o_strategy = model 
_ = play_a_game(x_strategy=x_strategy, o_strategy=o_strategy, verbose=True)


* We see that the game tends toward a draw, which is what we would expect from a couple of human players. Still, neither player is particularly good.

* As a final measure of skill, let's see how the length of the average game has changed. We would expect this to go down with an increased imbalance in skill levels.

In [ ]:
# @title Compare the lengths of the games

def avg_game_length(games, prec=1):
    lengths_as_floats = [float(len(game)) for game in games]
    avg = np.mean(lengths_as_floats)
    return round(avg, prec)

print(f"Average length of fully random game is {avg_game_length(games)} moves")
print(f"Average length of game where x uses NN is {avg_game_length(games3)} moves")
print(f"Average length of game where o uses NN is {avg_game_length(games4)} moves")
print(f"Average length of game where both use NN is {avg_game_length(games5} moves")


* As shown above, the games are a move shorter for player 1 and a bit longer for player 2.

* Now, we play a game against our model and see how well it does. We let it make the first move.

* We choose a DNN architecture since we effectively want to predict the outcome of a game based on the given board state.
* The input for each cell is the board state, which we reshape into a flat array of 9 elements, each element of which can be a 0 (empty cell), 1 (player 1 move), or 2 (player 2 move).
* The output is the result of the game (win, loss, or draw). We use a one-hot encoded array for this.

In [ ]:
# @title A human against the model

x_strategy = human_strategy
o_strategy = model
play_a_game(x_strategy=x_strategy, o_strategy=o_strategy, rnd=0.6)
print()

In [ ]:
# @title A human against the model

x_strategy = model
o_strategy = human_strategy 
play_a_game(x_strategy=x_strategy, o_strategy=o_strategy, rnd=0.6)
print()